In [14]:
import os
import json

import pandas as pd
from fastapi import FastAPI
from py2neo import Graph

In [6]:
url = os.environ.get("NEO4J_URL", "bolt://localhost:7687")
user = os.environ.get("NEO4J_USER", "neo4j")
pswd = os.environ.get("NEO4J_PSWD", "password")
graph = Graph(url, auth=(user, pswd))

csv_file_name = "results_full.csv"

In [48]:
response = graph.run(
    f"""
MATCH (n:package)
WHERE n.license CONTAINS "GNU general public license" OR n.license CONTAINS "GNU Affero general public license"

MATCH (n)<-[:DEPENDS_ON*1..]-(m)
WHERE NOT m.license CONTAINS "GNU general public license" AND NOT m.license CONTAINS "GNU Affero general public license"
RETURN m.name as name, m.license AS license, collect( DISTINCT n.name) as GPL_dependencies ORDER BY license
    """
).data()
all_miss_matches = pd.DataFrame(response)
all_miss_matches.to_csv(csv_file_name)

In [49]:
all_miss_matches = pd.read_csv(csv_file_name)
all_miss_matches

,Unnamed: 0,name,license,GPL_dependencies
0,0,frutils,"""the parity public license 6.0.0""",['cursor']
1,1,frkl,"""the parity public license 6.0.0""",['cursor']
2,2,ting,"""the parity public license 6.0.0""",['cursor']
3,3,freckles,"""the parity public license 6.0.0""",['cursor']
4,4,freckles_adapter_nsbl,"""the parity public license 6.0.0""",['cursor']
...,...,...,...,...
8752,8752,archetypes.referencebrowserwidget,zope public license,['plone.app.jquerytools']
8753,8753,Products.PlonePAS,zope public license,"['plone.base', 'plone.batching', 'plone.i18n',..."
8754,8754,five.customerize,zope public license,['plone.portlets']
8755,8755,Products.ZMySQLDA,zope public license,['mysqlclient']


In [51]:
all_miss_matches.license.value_counts().to_dict()

{'MIT license': 4020,
 'unknown': 2217,
 'Apache license': 755,
 'BSD license': 510,
 'GNU lesser general public license v3 (lgplv3)': 262,
 'Apache license 2.0': 220,
 'other/proprietary license': 175,
 'gnu lesser general public license': 73,
 'BSD 3-clause license': 40,
 'Mozilla public license 2.0 (mpl 2.0)': 39,
 'cc0 1.0 universal (cc0 1.0) public domain dedication': 37,
 'polyform shield license 1.0.0, https://polyformproject.org/wp-content/uploads/2020/06/polyform-shield-1.0.0.txt': 35,
 'academic free license (afl)': 25,
 'ISC license (iscl)': 24,
 'public domain': 22,
 'freeware': 19,
 'license.txt': 17,
 'the unlicense (unlicense)': 15,
 'GNU lesser general public license v2 (lgplv2)': 13,
 'free to use but restricted': 11,
 '"the parity public license 6.0.0"': 10,
 'european union public licence 1.2 (eupl 1.2)': 9,
 'license': 7,
 'lgplv3': 7,
 'Python software foundation license': 6,
 'bsd-3': 6,
 'gnu general public license version 3, 29 june 2007': 5,
 'creative commons 